In [38]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
import sys
sys.path.append('../Src/')

import os
import argparse
import random
import logging
import torch

import numpy as np
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from torchvision import transforms
from torch.utils.data import DataLoader,Dataset
from pathlib import Path

#from utils import __balance_val_split, __split_of_train_sequence, __log_class_statistics
from czech_slr_dataset import CzechSLRDataset,GaussianNoise
#from spoter_model import SPOTER
#from spoter.utils import train_epoch, evaluate
#from spoter.gaussian_noise import GaussianNoise

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [40]:
class args:
    def __init__(self):
        self.seed = 379
        self.testing_set_path = '../../DATASETS/AEC--openpose-val.csv'
        self.training_set_path = '../../DATASETS/AEC--openpose-train.csv' 
        self.experiment_name = 'cris_openpose_AEC'
        
        self.gaussian_mean  = 0
        self.gaussian_std  =0.001
                        
args = args()

In [4]:
random.seed(args.seed)
np.random.seed(args.seed)
os.environ["PYTHONHASHSEED"] = str(args.seed)
torch.manual_seed(args.seed)
torch.cuda.manual_seed(args.seed)
torch.cuda.manual_seed_all(args.seed)
torch.backends.cudnn.deterministic = True
g = torch.Generator()
g.manual_seed(args.seed)

In [5]:
# Set device to CUDA only if applicable
device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")

In [44]:
transform    = transforms.Compose([GaussianNoise(args.gaussian_mean, args.gaussian_std)])

train_set    = CzechSLRDataset(args.training_set_path, transform=transform, augmentations=False)
train_loader = DataLoader(train_set, shuffle=True, generator=g)

set labels
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]
BODY_IDENTIFIERS ['k0', 'k16', 'k15', 'k5', 'k2', 'k6', 'k3', 'k7', 'k4', 'k75', 'k46', 'k73', 'k42', 'k81', 'k44', 'k61', 'k28', 'k64', 'k32', 'k30', 'k77', 'k47', 'k79', 'k51', 'k83', 'k49', 'k70', 'k38', 'k67', 'k34', 'k36', 'k96', 'k97', 'k98', 'k99', 'k100', 'k101', 'k102', 'k103', 'k104', 'k105', 'k106', 'k107', 'k108', 'k109', 'k110', 'k111', 'k112', 'k113', 'k114', 'k115', 'k117', 'k118', 'k119', 'k120', 'k121', 'k122']
HAND_IDENTIFIERS ['k123', 'k124', 'k125', 'k126', 'k127', 'k128', 'k129', 'k130', 'k131', 'k132', 'k133', 'k134', 'k135', 'k136']


In [48]:
train_loader.dataset[130][1]

tensor([17.])

In [7]:
eval_set    = CzechSLRDataset(args.testing_set_path)
eval_loader = DataLoader(eval_set, shuffle=True, generator=g)

set labels
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]
BODY_IDENTIFIERS ['k0', 'k16', 'k15', 'k5', 'k2', 'k6', 'k3', 'k7', 'k4', 'k75', 'k46', 'k73', 'k42', 'k81', 'k44', 'k61', 'k28', 'k64', 'k32', 'k30', 'k77', 'k47', 'k79', 'k51', 'k83', 'k49', 'k70', 'k38', 'k67', 'k34', 'k36', 'k96', 'k97', 'k98', 'k99', 'k100', 'k101', 'k102', 'k103', 'k104', 'k105', 'k106', 'k107', 'k108', 'k109', 'k110', 'k111', 'k112', 'k113', 'k114', 'k115', 'k117', 'k118', 'k119', 'k120', 'k121', 'k122']
HAND_IDENTIFIERS ['k123', 'k124', 'k125', 'k126', 'k127', 'k128', 'k129', 'k130', 'k131', 'k132', 'k133', 'k134', 'k135', 'k136']


In [8]:
train_loader.dataset[3][0].shape

torch.Size([12, 71, 2])

In [9]:
type(train_loader.dataset[0])

tuple

In [10]:
import h5py
import glob
import pandas as pd
import tqdm
import time
import gc
from collections import Counter

    
paths = glob.glob('../../DATASETS/*.hdf5')

In [11]:
def get_data_from_h5(path):
    hf = h5py.File(path, 'r')
    return hf

In [12]:
for path in paths:
    print('reading :',path)
    data = get_data_from_h5(path)
    #torch.Size([5, 71, 2])
    data_video = np.array(data['0']['data'])
    data_label = np.array(data['0']['label']).item().decode('utf-8')
    #data = data_video.reshape((12,137,2))
    print(data_video.shape,data_label)

reading : ../../DATASETS/AEC--mediapipe-Train.hdf5
(14, 2, 543) aprender
reading : ../../DATASETS/AEC--mediapipe-Val.hdf5
(17, 2, 543) hacer
reading : ../../DATASETS/AEC--openpose-Train.hdf5
(11, 2, 137) aprender
reading : ../../DATASETS/AEC--openpose-Val.hdf5
(12, 2, 137) hacer
reading : ../../DATASETS/AEC--openpose.hdf5
(12, 2, 137) antes
reading : ../../DATASETS/AEC--wholepose-Train.hdf5
(14, 2, 133) aprender
reading : ../../DATASETS/AEC--wholepose-Val.hdf5
(17, 2, 133) hacer
reading : ../../DATASETS/PUCP_PSL_DGI156--mediapipe-Train.hdf5
(43, 2, 543) mujer
reading : ../../DATASETS/PUCP_PSL_DGI156--mediapipe-Val.hdf5
(22, 2, 543) mamá
reading : ../../DATASETS/PUCP_PSL_DGI156--openpose-Train.hdf5
(9, 2, 137) mujer
reading : ../../DATASETS/PUCP_PSL_DGI156--openpose-Val.hdf5
(12, 2, 137) mamá
reading : ../../DATASETS/PUCP_PSL_DGI156--wholepose-Train.hdf5
(43, 2, 133) mujer
reading : ../../DATASETS/PUCP_PSL_DGI156--wholepose-Val.hdf5
(22, 2, 133) mamá
reading : ../../DATASETS/WLASL--medi

In [13]:

def get_dataset_from_hdf5(path,keypoints_model,threshold_frecuency_labels=10,list_labels_banned=[]):

    index_array_column = None #'mp_indexInArray', 'wp_indexInArray','op_indexInArray'

    print('Use keypoint model : ',keypoints_model) 
    if keypoints_model == 'openpose':
        index_array_column  = 'op_indexInArray'
    if keypoints_model == 'mediapipe':
        index_array_column  = 'mp_indexInArray'
    if keypoints_model == 'wholepose':
        index_array_column  = 'wp_indexInArray'
    print('use column for index keypoint :',index_array_column)

    assert not index_array_column is None

    df_keypoints = pd.read_csv('../../DATASETS/Mapeo landmarks librerias - Hoja 1.csv', skiprows=1)
    df_keypoints = df_keypoints[(df_keypoints['Selected73']=='x' )& (df_keypoints['Key']!='wrist')]
    idx_keypoints = sorted(df_keypoints[index_array_column].astype(int).values)
    name_keypoints = df_keypoints['Key'].values
    section_keypoints = (df_keypoints['Section']+'_'+df_keypoints['Key']).values
    print('section_keypoints : ',len(section_keypoints),' -- uniques: ',len(set(section_keypoints)))
    print('name_keypoints    : ',len(name_keypoints),' -- uniques: ',len(set(name_keypoints)))
    print('idx_keypoints     : ',len(idx_keypoints),' -- uniques: ',len(set(idx_keypoints)))
    print('')
    print('section_keypoints used:')
    print(section_keypoints)


    print('Reading dataset .. ')
    data = get_data_from_h5(path)
    #torch.Size([5, 71, 2])
    print('Total size dataset : ',len(data.keys()))
    video_dataset  = []
    labels_dataset = []

    time.sleep(2)
    for index in tqdm.tqdm(list(data.keys())):

        data_video = np.array(data[index]['data'])
        data_label = np.array(data[index]['label']).item().decode('utf-8')

        n_frames,n_axis,n_keypoints = data_video.shape

        data_video = data_video.reshape((n_frames,n_keypoints,n_axis))
        if index=='0':
            print('original size video : ',data_video.shape,'-- label : ',data_label)
            print('filtering by keypoints idx .. ')
        data_video = data_video[:,idx_keypoints,:]
        if index=='0':
            print('filtered size video : ',data_video.shape,'-- label : ',data_label)

        video_dataset.append(data_video)
        labels_dataset.append(data_label)
    del data
    gc.collect()
    
    print('label encoding ...')
    dict_labels_dataset = {}
    inv_dict_labels_dataset = {}

    for index,label in enumerate(set(labels_dataset)):
        dict_labels_dataset[label] = index
        inv_dict_labels_dataset[index] = label
        
    encoded_dataset = [dict_labels_dataset[label] for label in labels_dataset]
    
    print('label encoding completed!')
    
    print('frecuency labels filtering ...')
    hist_labels = dict(Counter(labels_dataset))
    labels_high_frecuency = []
    for name in hist_labels.keys():
        if hist_labels[name] >= threshold_frecuency_labels and not name in list_labels_banned:
            labels_high_frecuency.append(name)
    labels_high_frecuency = sorted(labels_high_frecuency)
    len(labels_high_frecuency)
    len(set(labels_dataset))
    filtros = [label in labels_high_frecuency for label in labels_dataset]
    
    print('before filter size video_dataset   :',len(video_dataset))
    print('before filter size labels_dataset  :',len(labels_dataset))
    print('before filter size encoded_dataset :',len(encoded_dataset))
    video_dataset   = np.array(video_dataset)[filtros]
    labels_dataset  = np.array(labels_dataset)[filtros]
    encoded_dataset = np.array(encoded_dataset)[filtros]
    print('after  filter size video_dataset   :',len(video_dataset))
    print('after  filter size labels_dataset  :',len(labels_dataset))
    print('after  filter size encoded_dataset :',len(encoded_dataset))
    print('frecuency labels completed!')

    print('Reading dataset completed!')
    
    return video_dataset,labels_dataset,encoded_dataset,dict_labels_dataset,inv_dict_labels_dataset

In [14]:
path = '../../DATASETS/AEC--openpose-Train.hdf5'
keypoints_model = 'openpose' # mediapipe, wholepose

video_dataset,labels_dataset,encoded_dataset,dict_labels_dataset,inv_dict_labels_dataset = get_dataset_from_hdf5(path,keypoints_model)

Use keypoint model :  openpose
use column for index keypoint : op_indexInArray
section_keypoints :  71  -- uniques:  71
name_keypoints    :  71  -- uniques:  51
idx_keypoints     :  71  -- uniques:  71

section_keypoints used:
['pose_nose' 'pose_left_eye' 'pose_right_eye' 'pose_left_shoulder'
 'pose_right_shoulder' 'pose_left_elbow' 'pose_right_elbow'
 'pose_left_wrist' 'pose_right_wrist' 'face_right_mouth_up'
 'face_right_eyebrow_inner' 'face_right_mouth_corner'
 'face_right_eyebrow_outer' 'face_right_mouth_down'
 'face_right_eyebrow_middle' 'face_right_eye_outer' 'face_right_jaw_up'
 'face_right_eye_inner' 'face_right_jaw_down' 'face_right_jaw_middle'
 'face_left_mouth_up' 'face_left_eyebrow_inner' 'face_left_mouth_corner'
 'face_left_eyebrow_outer' 'face_left_mouth_down'
 'face_left_eyebrow_middle' 'face_left_eye_outer' 'face_left_jaw_up'
 'face_left_eye_inner' 'face_left_jaw_down' 'face_left_jaw_middle'
 'leftHand_thumb_cmc' 'leftHand_thumb_mcp' 'leftHand_thumb_ip'
 'leftHand_thumb

 61%|███████████████████████▋               | 380/625 [00:00<00:00, 1637.12it/s]

original size video :  (11, 137, 2) -- label :  aprender
filtering by keypoints idx .. 
filtered size video :  (11, 71, 2) -- label :  aprender


100%|███████████████████████████████████████| 625/625 [00:00<00:00, 2035.55it/s]

label encoding ...
label encoding completed!
frecuency labels filtering ...
before filter size video_dataset   : 625
before filter size labels_dataset  : 625
before filter size encoded_dataset : 625
after  filter size video_dataset   : 625
after  filter size labels_dataset  : 625
after  filter size encoded_dataset : 625
frecuency labels completed!
Reading dataset completed!



/tmp/ipykernel_12419/1576412037.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  video_dataset   = np.array(video_dataset)[filtros]


In [30]:
class LSP_Dataset(Dataset):
    """Advanced object representation of the HPOES dataset for loading hand joints landmarks utilizing the Torch's
    built-in Dataset properties"""

    data: [np.ndarray]  # type: ignore
    labels: [np.ndarray]  # type: ignore

    def __init__(self, dataset_filename: str,keypoints_model:str, num_labels=5, transform=None, augmentations=False,
                 augmentations_prob=0.5, normalize=False):
        """
        Initiates the HPOESDataset with the pre-loaded data from the h5 file.

        :param dataset_filename: Path to the h5 file
        :param transform: Any data transformation to be applied (default: None)
        """

        self.list_labels_banned = ["ya", "qué?", "qué", "bien", "dos", "ahí", "luego", "yo", "él", "tú","???","NNN"]

        video_dataset,labels_dataset,encoded_dataset,dict_labels_dataset,inv_dict_labels_dataset = get_dataset_from_hdf5(dataset_filename,keypoints_model,
                                                                                         threshold_frecuency_labels =10,
                                                                                         list_labels_banned =self.list_labels_banned)

        self.data = video_dataset
        self.labels = encoded_dataset
        #self.targets = list(encoded_dataset)
        self.text_labels = list(labels_dataset)
        self.num_labels = num_labels
        self.transform = transform

        self.augmentations = augmentations
        self.augmentations_prob = augmentations_prob
        self.normalize = normalize

    def __getitem__(self, idx):
        """
        Allocates, potentially transforms and returns the item at the desired index.

        :param idx: Index of the item
        :return: Tuple containing both the depth map and the label
        """
        depth_map = torch.from_numpy(np.copy(self.data[idx]))
        label = torch.Tensor([self.labels[idx] - 1])
        depth_map = depth_map - 0.5
        if self.transform:
            depth_map = self.transform(depth_map)
        return depth_map, label

    def __len__(self):
        return len(self.labels)

In [16]:
path = '../../DATASETS/AEC--openpose-Train.hdf5'
keypoints_model = 'openpose' # mediapipe, wholepose

transform    = transforms.Compose([GaussianNoise(args.gaussian_mean, args.gaussian_std)])

train_set    = LSP_Dataset(path,keypoints_model, transform=transform, augmentations=False)
train_loader = DataLoader(train_set, shuffle=True, generator=g)

Use keypoint model :  openpose
use column for index keypoint : op_indexInArray
section_keypoints :  71  -- uniques:  71
name_keypoints    :  71  -- uniques:  51
idx_keypoints     :  71  -- uniques:  71

section_keypoints used:
['pose_nose' 'pose_left_eye' 'pose_right_eye' 'pose_left_shoulder'
 'pose_right_shoulder' 'pose_left_elbow' 'pose_right_elbow'
 'pose_left_wrist' 'pose_right_wrist' 'face_right_mouth_up'
 'face_right_eyebrow_inner' 'face_right_mouth_corner'
 'face_right_eyebrow_outer' 'face_right_mouth_down'
 'face_right_eyebrow_middle' 'face_right_eye_outer' 'face_right_jaw_up'
 'face_right_eye_inner' 'face_right_jaw_down' 'face_right_jaw_middle'
 'face_left_mouth_up' 'face_left_eyebrow_inner' 'face_left_mouth_corner'
 'face_left_eyebrow_outer' 'face_left_mouth_down'
 'face_left_eyebrow_middle' 'face_left_eye_outer' 'face_left_jaw_up'
 'face_left_eye_inner' 'face_left_jaw_down' 'face_left_jaw_middle'
 'leftHand_thumb_cmc' 'leftHand_thumb_mcp' 'leftHand_thumb_ip'
 'leftHand_thumb

 63%|████████████████████████▌              | 394/625 [00:00<00:00, 1762.70it/s]

original size video :  (11, 137, 2) -- label :  aprender
filtering by keypoints idx .. 
filtered size video :  (11, 71, 2) -- label :  aprender


100%|███████████████████████████████████████| 625/625 [00:00<00:00, 2091.38it/s]


label encoding ...
label encoding completed!
frecuency labels filtering ...
before filter size video_dataset   : 625
before filter size labels_dataset  : 625
before filter size encoded_dataset : 625
after  filter size video_dataset   : 625
after  filter size labels_dataset  : 625
after  filter size encoded_dataset : 625
frecuency labels completed!
Reading dataset completed!


/tmp/ipykernel_12419/1576412037.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  video_dataset   = np.array(video_dataset)[filtros]


In [17]:
train_loader.dataset[0][0].shape

torch.Size([11, 71, 2])

In [43]:
train_loader.dataset[10][1].shape

torch.Size([1])

In [41]:
from Lsp_dataset import LSP_Dataset as test_loader

In [49]:
path = '../../DATASETS/AEC--openpose-Train.hdf5'
keypoints_model = 'openpose' # mediapipe, wholepose

transform    = transforms.Compose([GaussianNoise(args.gaussian_mean, args.gaussian_std)])

train_set    = test_loader(path,keypoints_model, transform=transform, augmentations=False,landmarks_ref= '../../DATASETS/Mapeo landmarks librerias - Hoja 1.csv')
train_loader = DataLoader(train_set, shuffle=True, generator=g)

Use keypoint model :  openpose
use column for index keypoint : op_indexInArray
section_keypoints :  71  -- uniques:  71
name_keypoints    :  71  -- uniques:  51
idx_keypoints     :  71  -- uniques:  71

section_keypoints used:
['pose_nose' 'pose_left_eye' 'pose_right_eye' 'pose_left_shoulder'
 'pose_right_shoulder' 'pose_left_elbow' 'pose_right_elbow'
 'pose_left_wrist' 'pose_right_wrist' 'face_right_mouth_up'
 'face_right_eyebrow_inner' 'face_right_mouth_corner'
 'face_right_eyebrow_outer' 'face_right_mouth_down'
 'face_right_eyebrow_middle' 'face_right_eye_outer' 'face_right_jaw_up'
 'face_right_eye_inner' 'face_right_jaw_down' 'face_right_jaw_middle'
 'face_left_mouth_up' 'face_left_eyebrow_inner' 'face_left_mouth_corner'
 'face_left_eyebrow_outer' 'face_left_mouth_down'
 'face_left_eyebrow_middle' 'face_left_eye_outer' 'face_left_jaw_up'
 'face_left_eye_inner' 'face_left_jaw_down' 'face_left_jaw_middle'
 'leftHand_thumb_cmc' 'leftHand_thumb_mcp' 'leftHand_thumb_ip'
 'leftHand_thumb

 77%|█████████████████████████████▉         | 479/625 [00:00<00:00, 2397.55it/s]

original size video :  (11, 137, 2) -- label :  aprender
filtering by keypoints idx .. 
filtered size video :  (11, 71, 2) -- label :  aprender


100%|███████████████████████████████████████| 625/625 [00:00<00:00, 2338.73it/s]


label encoding ...
label encoding completed!
frecuency labels filtering ...
hist counter
{'aprender': 18, 'hacer': 14, 'tres': 17, 'proteína': 30, 'sí': 36, 'cómo': 19, 'comer': 58, 'cuánto': 36, 'porcentaje': 44, 'dentro': 14, 'decir': 17, 'cero': 24, 'ahora': 15, 'hermano': 26, 'no': 22, 'pensar': 25, 'niño': 12, 'manejar': 12, 'ver': 18, 'G-R': 27, 'uno': 18, 'emoción': 23, 'ese': 19, 'cien': 24, 'fuerte': 17, 'solucionar': 14, 'conflicto': 13, 'importante': 13}
total unique labels :  28
before filter size video_dataset   : 625
before filter size labels_dataset  : 625
before filter size encoded_dataset : 625
after  filter size video_dataset   : 625
after  filter size labels_dataset  : 625
after  filter size encoded_dataset : 625
frecuency labels completed!
Reading dataset completed!


/media/cristian/Extreme/Investigacion/SL/Spoter-SL/Codes/../Src/Lsp_dataset.py:105: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  print('Reading dataset completed!')


In [51]:
train_loader.dataset[130][1]

tensor([14.])